In [1]:
# https://github.com/huggingface/lerobot/blob/main/examples/7_get_started_with_real_robot.md

In [1]:
import os
import sys
parent_of_parent = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_of_parent)

In [2]:
from lerobot.common.robot_devices.motors.configs import DynamixelMotorsBusConfig
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus

leader_port = "/dev/ttyACM5"
follower_port = "/dev/ttyACM2"

leader_config = DynamixelMotorsBusConfig(
    port=leader_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),
    },
)

follower_config = DynamixelMotorsBusConfig(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

leader_arm = DynamixelMotorsBus(leader_config)
follower_arm = DynamixelMotorsBus(follower_config)

teleop

In [3]:
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

robot_config = KochRobotConfig(
    leader_arms={"main": leader_config},
    follower_arms={"main": follower_config},
    cameras={},  # We don't use any camera for now
)
robot = ManipulatorRobot(robot_config)

In [4]:
# calibrate robot to make sure leader and follower are in the same position
robot.connect()

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [5]:
# calibration complete! check if match
leader_pos = robot.leader_arms["main"].read("Present_Position")
follower_pos = robot.follower_arms["main"].read("Present_Position")

print(leader_pos)
print(follower_pos)

[ 14.238281 120.67383  163.38867   39.46289   80.24414   34.716797]
[-15.292969 126.47461  161.80664   28.03711   73.916016  34.277344]


teleop for 30s

In [11]:
import tqdm
seconds = 30
frequency = 200 # 200 Hz
for _ in tqdm.tqdm(range(seconds*frequency)):
    leader_pos = robot.leader_arms["main"].read("Present_Position")
    robot.follower_arms["main"].write("Goal_Position", leader_pos)

100%|██████████| 6000/6000 [00:29<00:00, 204.21it/s]


In [15]:
# same but simpler code version
for _ in tqdm.tqdm(range(seconds*frequency)):
    robot.teleop_step()

100%|██████████| 6000/6000 [00:29<00:00, 202.53it/s]


In [20]:
leader_pos = robot.leader_arms["main"].read("Present_Position")
follower_pos = robot.follower_arms["main"].read("Present_Position")
# observation = follower arm original pos, action = leader arm pos
observation, action = robot.teleop_step(record_data=True)

print(follower_pos)
print(observation)
print(leader_pos)
print(action)

[-21.796875 125.771484 167.69531   20.126953  76.02539   34.277344]
{'observation.state': tensor([-21.7969, 125.7715, 167.6953,  20.1270,  76.0254,  34.2773])}
[-15.1171875 133.50586   162.59766    31.113281   73.47656    34.716797 ]
{'action': tensor([-15.1172, 133.5059, 162.5977,  31.1133,  73.4766,  34.7168])}


In [6]:
robot.disconnect()

teleop with camera (03_camera.ipynb)

In [10]:
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera

# check camera index corresponding to OBS
# python lerobot/common/robot_devices/cameras/opencv.py --images-dir outputs/images_from_opencv_cameras
camera_config = OpenCVCameraConfig(camera_index=4)
camera = OpenCVCamera(camera_config)
camera.connect()
color_image = camera.read()

print(color_image.shape)
print(color_image.dtype)

(480, 640, 3)
uint8


In [13]:
# MAKE SURE TO RUN THIS BEFORE TRYING TO ASSIGN IN CONFIG
camera.disconnect()

RobotDeviceNotConnectedError: OpenCVCamera(4) is not connected. Try running `camera.connect()` first.

In [16]:
robot_config = KochRobotConfig(
    leader_arms={"main": leader_config},
    follower_arms={"main": follower_config},
    calibration_dir=".cache/calibration/koch",
    cameras={
        # "laptop": OpenCVCameraConfig(0, fps=30, width=640, height=480),
        "phone": OpenCVCameraConfig(4, fps=30, width=640, height=480),
    },
)
robot = ManipulatorRobot(robot_config)
robot.connect()

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [17]:
observation, action = robot.teleop_step(record_data=True)
# print(observation["observation.images.laptop"].shape)
print(observation["observation.images.phone"].shape)
print(observation["observation.images.phone"].min().item())
print(observation["observation.images.phone"].max().item())


torch.Size([480, 640, 3])
0
255


In [19]:
robot.disconnect()

record data and visualize it

In [21]:
robot.connect()

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [22]:
import time
from lerobot.scripts.control_robot import busy_wait

record_time_s = 30
fps = 60

states = []
actions = []
for _ in range(record_time_s * fps):
    start_time = time.perf_counter()
    observation, action = robot.teleop_step(record_data=True)

    states.append(observation["observation.state"])
    actions.append(action["action"])

    dt_s = time.perf_counter() - start_time
    busy_wait(1 / fps - dt_s)

# Note that observation and action are available in RAM, but
# you could potentially store them on disk with pickle/hdf5 or
# our optimized format `LeRobotDataset`. More on this next.

Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error reading in thread: Can't capture color image from camera 4.
Error read